In [1]:
import rdflib
import time
import pathlib
from dotenv import load_dotenv
import os

from settings import path_base

In [2]:
path_to_secrets = pathlib.Path(path_base, 'secrets.env')
try:
    load_dotenv(dotenv_path=path_to_secrets)  # Load secrets/env variables
except:
    print('secrets could not be loaded!')

In [3]:
#utility function to get the local part of a URI (stripping out the namespace)

def getLocalPart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[pos+1:]

def getNamespacePart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[0:pos+1]

# quick test
print(getLocalPart("http://onto.neo4j.com/rail#Station"))
print(getNamespacePart("http://onto.neo4j.com/rail#Station"))

Station
http://onto.neo4j.com/rail#


In [4]:
# Own ontology:
# path_to_ontology = pathlib.Path(path_base, "models/Ontologies/Ontology3.ttl").as_posix()
path_to_ontology = pathlib.Path(path_base, "models/Ontologies/rail.ttl").as_posix()
# From tutorial:
# path_to_ontology = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl"

In [5]:
g = rdflib.Graph()
g.parse(source=path_to_ontology, format='turtle')

simple_query = """
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?c
  WHERE {
    ?c rdf:type owl:Class .
  } """

for row in g.query(simple_query):
    print('URI: ', str(row.c),'\nCLASS: ',getLocalPart(str(row.c)),'\nNAMESPACE: ',getNamespacePart(str(row.c)))
    # print(str(row.c))
    # print(str(getLocalPart(str(row.c))))
    print('-----------------------------------------------------------')


URI:  http://onto.neo4j.com/rail#Event 
CLASS:  Event 
NAMESPACE:  http://onto.neo4j.com/rail#
-----------------------------------------------------------
URI:  http://onto.neo4j.com/rail#Station 
CLASS:  Station 
NAMESPACE:  http://onto.neo4j.com/rail#
-----------------------------------------------------------


In [6]:
# read the onto and generate cypher (complete without mappings)

g = rdflib.Graph()
g.parse(path_to_ontology, format='turtle')

classes_and_props_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
WHERE {
    ?curi rdf:type owl:Class .
    optional { 
      ?prop rdfs:domain ?curi ;
        a owl:DatatypeProperty ;
        rdfs:range ?range .
      BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
    }
  } GROUP BY ?curi  """

query_result_classes_and_props = g.query(classes_and_props_query)
print(query_result_classes_and_props.vars)

[rdflib.term.Variable('curi'), rdflib.term.Variable('props')]


In [7]:
for binding in query_result_classes_and_props.bindings:
    print(binding)
    print('-------------')

{rdflib.term.Variable('curi'): rdflib.term.URIRef('http://onto.neo4j.com/rail#Event'), rdflib.term.Variable('props'): rdflib.term.Literal('http://onto.neo4j.com/rail#eventDescription;http://www.w3.org/2001/XMLSchema#string,http://onto.neo4j.com/rail#eventId;http://www.w3.org/2001/XMLSchema#string,http://onto.neo4j.com/rail#eventType;http://www.w3.org/2001/XMLSchema#string')}
-------------
{rdflib.term.Variable('curi'): rdflib.term.URIRef('http://onto.neo4j.com/rail#Station'), rdflib.term.Variable('props'): rdflib.term.Literal('http://onto.neo4j.com/rail#lat;http://www.w3.org/2001/XMLSchema#float,http://onto.neo4j.com/rail#long;http://www.w3.org/2001/XMLSchema#float,http://onto.neo4j.com/rail#stationAddress;http://www.w3.org/2001/XMLSchema#string,http://onto.neo4j.com/rail#stationCode;http://www.w3.org/2001/XMLSchema#string,http://onto.neo4j.com/rail#stationName;http://www.w3.org/2001/XMLSchema#string')}
-------------


In [8]:
cypher_list = []

for row in query_result_classes_and_props:
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("merge (n:" + getLocalPart(row.curi) + " { `<id_prop>`: record.`<col with id>`} )")
    # print('----- OUTER -----')
    # print('row.curi:', row.curi)
    # print('     ----- inner ------')
    # print('row.props:', row.props)
    for pair in row.props.split(","):
        propName = pair.split(";")[0]
        # print('propName:', propName)
        propType = pair.split(";")[1]
        # print('propType:', propType)
        cypher.append("set n." + getLocalPart(propName) + " = record.`<col with value for " + getLocalPart(propName) + ">`")
        # print('getLocalPart(propName):', getLocalPart(propName))
        # print('     ----- inner ------')
    # print('----- OUTER -----')
    cypher.append("return count(*) as total") 
    cypher_list.append(' \n'.join(cypher))

for cypher in cypher_list:
    print(cypher)
    print('-----------')

unwind $records AS record 
merge (n:Event { `<id_prop>`: record.`<col with id>`} ) 
set n.eventDescription = record.`<col with value for eventDescription>` 
set n.eventId = record.`<col with value for eventId>` 
set n.eventType = record.`<col with value for eventType>` 
return count(*) as total
-----------
unwind $records AS record 
merge (n:Station { `<id_prop>`: record.`<col with id>`} ) 
set n.lat = record.`<col with value for lat>` 
set n.long = record.`<col with value for long>` 
set n.stationAddress = record.`<col with value for stationAddress>` 
set n.stationCode = record.`<col with value for stationCode>` 
set n.stationName = record.`<col with value for stationName>` 
return count(*) as total
-----------


In [9]:
rels_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
WHERE {
    ?rel a ?propertyClass .
    filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
           owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
    
    ?rel rdfs:domain ?dom ;
      rdfs:range ?ran .
    
    #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
    
  }"""

query_result_relations = g.query(rels_query)
print(query_result_relations.vars)

[rdflib.term.Variable('rel'), rdflib.term.Variable('dom'), rdflib.term.Variable('ran')]


In [10]:
for row in query_result_relations:
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("match (source:" + getLocalPart(row.dom) + " { `<id_prop>`: record.`<col with source id>`} )")
    cypher.append("match (target:" + getLocalPart(row.ran) + " { `<id_prop>`: record.`<col with target id>`} )")
    cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
    cypher.append("return count(*) as total") 
    for item in cypher:
        print('cypher-item:\n', item)
    print('---------------------------------------')
    cypher_list.append(' \n'.join(cypher))


cypher-item:
 unwind $records AS record
cypher-item:
 match (source:Event { `<id_prop>`: record.`<col with source id>`} )
cypher-item:
 match (target:Station { `<id_prop>`: record.`<col with target id>`} )
cypher-item:
 merge (source)-[r:`affects`]->(target)
cypher-item:
 return count(*) as total
---------------------------------------
cypher-item:
 unwind $records AS record
cypher-item:
 match (source:Station { `<id_prop>`: record.`<col with source id>`} )
cypher-item:
 match (target:Station { `<id_prop>`: record.`<col with target id>`} )
cypher-item:
 merge (source)-[r:`link`]->(target)
cypher-item:
 return count(*) as total
---------------------------------------


In [11]:
for q in cypher_list:
    print("\n\n" + q)



unwind $records AS record 
merge (n:Event { `<id_prop>`: record.`<col with id>`} ) 
set n.eventDescription = record.`<col with value for eventDescription>` 
set n.eventId = record.`<col with value for eventId>` 
set n.eventType = record.`<col with value for eventType>` 
return count(*) as total


unwind $records AS record 
merge (n:Station { `<id_prop>`: record.`<col with id>`} ) 
set n.lat = record.`<col with value for lat>` 
set n.long = record.`<col with value for long>` 
set n.stationAddress = record.`<col with value for stationAddress>` 
set n.stationCode = record.`<col with value for stationCode>` 
set n.stationName = record.`<col with value for stationName>` 
return count(*) as total


unwind $records AS record 
match (source:Event { `<id_prop>`: record.`<col with source id>`} ) 
match (target:Station { `<id_prop>`: record.`<col with target id>`} ) 
merge (source)-[r:`affects`]->(target) 
return count(*) as total


unwind $records AS record 
match (source:Station { `<id_prop>`

In [12]:
railMappings = {}

stationMapping = {}
stationMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-stations-all.csv"
stationMapping["@uniqueId"] = "stationCode"
stationMapping["lat"] = "lat"
stationMapping["long"] = "long"
stationMapping["stationAddress"] = "address"
stationMapping["stationCode"] = "crs"
stationMapping["stationName"] = "name"
railMappings["Station"] = stationMapping

eventMapping = {}
eventMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv"
eventMapping["@uniqueId"] = "eventId"
eventMapping["eventDescription"] = "desc"
eventMapping["eventId"] = "id"
eventMapping["timestamp"] = "ts"
eventMapping["eventType"] = "type"
railMappings["Event"] = eventMapping

linkMapping = {}
linkMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-station-links.csv"
linkMapping["@from"] = "origin"
linkMapping["@to"] = "destination"
railMappings["link"] = linkMapping

affectsMapping = {}
affectsMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv"
affectsMapping["@from"] = "id"
affectsMapping["@to"] = "Station"
railMappings["affects"] = affectsMapping

# show it?
railMappings
from pprint import pprint
pprint(railMappings)

{'Event': {'@fileName': 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv',
           '@uniqueId': 'eventId',
           'eventDescription': 'desc',
           'eventId': 'id',
           'eventType': 'type',
           'timestamp': 'ts'},
 'Station': {'@fileName': 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-stations-all.csv',
             '@uniqueId': 'stationCode',
             'lat': 'lat',
             'long': 'long',
             'stationAddress': 'address',
             'stationCode': 'crs',
             'stationName': 'name'},
 'affects': {'@fileName': 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv',
             '@from': 'id',
             '@to': 'Station'},
 'link': {'@fileName': 'https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-station-links.csv',
          '@from': 'origin',
          '@to': 'destination'}}


In [13]:
#copy of previous but using the mappings
def getLoadersFromOnto(onto, rdf_format, mappings):
    g = rdflib.Graph()
    g.parse(onto, format=rdf_format)

    classes_and_props_query = """
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    
    SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
    WHERE {
      ?curi rdf:type owl:Class .
      optional { 
        ?prop rdfs:domain ?curi ;
          a owl:DatatypeProperty ;
          rdfs:range ?range .
        BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
      }
    } GROUP BY ?curi  """
    
    cypher_import = {}
    export_ns = set()
    export_mappings = {}
    
    for row in g.query(classes_and_props_query):
        export_ns.add(getNamespacePart(row.curi))
        export_mappings[getLocalPart(row.curi)] = str(row.curi)
        cypher = []
        cypher.append("unwind $records AS record")
        cypher.append("merge (n:" + getLocalPart(row.curi) + " { `" + mappings[getLocalPart(row.curi)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.curi)][mappings[getLocalPart(row.curi)]["@uniqueId"]] + "`} )")
        print("classes_and_props_query:", "merge (n:" + getLocalPart(row.curi) + " { `" + mappings[getLocalPart(row.curi)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.curi)][mappings[getLocalPart(row.curi)]["@uniqueId"]] + "`} )")
        for pair in row.props.split(","):      
            propName = pair.split(";")[0]
            propType = pair.split(";")[1]
            export_ns.add(getNamespacePart(propName))
            export_mappings[getLocalPart(propName)] = propName
            #if a mapping (a column in the source file) is defined for the property and property is not a unique id
            if getLocalPart(propName) in mappings[getLocalPart(row.curi)] and getLocalPart(propName) != mappings[getLocalPart(row.curi)]["@uniqueId"]:
                cypher.append("set n." + getLocalPart(propName) + " = record.`" + mappings[getLocalPart(row.curi)][getLocalPart(propName)] + "`")
        cypher.append("return count(*) as total") 
        cypher_import[getLocalPart(row.curi)] = ' \n'.join(cypher)
        # print('CYPHER in classes_and_props_query:\n', cypher)


    rels_query = """
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    
    SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
    WHERE {
      ?rel a ?propertyClass .
      filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
            owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
      
      ?rel rdfs:domain ?dom ;
        rdfs:range ?ran .
      
      #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
      
    }"""

    for row in g.query(rels_query):
        export_ns.add(getNamespacePart(row.rel))
        export_mappings[getLocalPart(row.rel)] = str(row.rel)
        cypher = []
        cypher.append("unwind $records AS record")
        cypher.append("match (source:" + getLocalPart(row.dom) + " { `" + mappings[getLocalPart(row.dom)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@from"] + "`} )")
        print("rels_query:", "match (source:" + getLocalPart(row.dom) + " { `" + mappings[getLocalPart(row.dom)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@from"] + "`} )")
        cypher.append("match (target:" + getLocalPart(row.ran) + " { `" + mappings[getLocalPart(row.ran)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@to"] + "`} )")
        cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
        cypher.append("return count(*) as total") 
        cypher_import[getLocalPart(row.rel)] = ' \n'.join(cypher)
        # print('CYPHER in rels_query:\n', cypher)


    nscount = 0
    mapping_export_cypher = []
    
    for ns in export_ns:
        print('----- nsprefixes')
        print("call n10s.nsprefixes.add('ns" + str(nscount) + "','" + ns + "');")
        mapping_export_cypher.append("call n10s.nsprefixes.add('ns" + str(nscount) + "','" + ns + "');")
        nscount+=1
    
    for k in export_mappings.keys():
        mapping_export_cypher.append("call n10s.mapping.add('" + export_mappings[k] + "','" + k + "');")
    
    return cypher_import ,  mapping_export_cypher




In [18]:
cypher_import , mapping_defs = getLoadersFromOnto("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl","turtle",railMappings)

print("#LOADERS:\n\n")
for q in cypher_import.keys():
  print(q + ": \n\nfile: " + railMappings[q]["@fileName"] + "\n\n"+ cypher_import[q] + "\n\n")

print('---------------------------------------------------------------')

print("#EXPORT MAPPINGS (for RDF API):\n\n")
for md in mapping_defs:
  print(md)

classes_and_props_query: merge (n:Event { `eventId`: record.`id`} )
classes_and_props_query: merge (n:Station { `stationCode`: record.`crs`} )
rels_query: match (source:Event { `eventId`: record.`id`} )
rels_query: match (source:Station { `stationCode`: record.`origin`} )
----- nsprefixes
call n10s.nsprefixes.add('ns0','http://onto.neo4j.com/rail#');
#LOADERS:


Event: file: https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv unwind $records AS record 
merge (n:Event { `eventId`: record.`id`} ) 
set n.eventDescription = record.`desc` 
set n.eventType = record.`type` 
return count(*) as total


Station: file: https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-stations-all.csv unwind $records AS record 
merge (n:Station { `stationCode`: record.`crs`} ) 
set n.lat = record.`lat` 
set n.long = record.`long` 
set n.stationAddress = record.`address` 
set n.stationName = record.`name` 
return count(*) as total


affects: file: https://

In [15]:
# Utility function to write to Neo4j in batch mode.

def insert_data(session, query, frame, batch_size = 500):
    
    print('QUERY:\n', query)
    print('---------------------------')
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    
    
    while batch * batch_size < len(frame):
        res = session.write_transaction( lambda tx: tx.run(query,
                      parameters = {'records': frame[batch*batch_size:(batch+1)*batch_size].to_dict('records')}).data())

        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result


In [19]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth

uri = "neo4j://localhost:7687"
auth = ("neo4j", os.getenv('NEO4J_PW'))
driver = GraphDatabase.driver(uri, auth=auth)

session = driver.session(database="neo4j")

cypher_import , mapping_defs = getLoadersFromOnto("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl","turtle",railMappings)

from pprint import pprint
print('CYPHER_IMPORT:')
pprint(cypher_import)
print('----------------------------------')
pprint(mapping_defs)

classes_and_props_query: merge (n:Event { `eventId`: record.`id`} )
classes_and_props_query: merge (n:Station { `stationCode`: record.`crs`} )
rels_query: match (source:Event { `eventId`: record.`id`} )
rels_query: match (source:Station { `stationCode`: record.`origin`} )
----- nsprefixes
call n10s.nsprefixes.add('ns0','http://onto.neo4j.com/rail#');
CYPHER_IMPORT:
{'Event': 'unwind $records AS record \n'
          'merge (n:Event { `eventId`: record.`id`} ) \n'
          'set n.eventDescription = record.`desc` \n'
          'set n.eventType = record.`type` \n'
          'return count(*) as total',
 'Station': 'unwind $records AS record \n'
            'merge (n:Station { `stationCode`: record.`crs`} ) \n'
            'set n.lat = record.`lat` \n'
            'set n.long = record.`long` \n'
            'set n.stationAddress = record.`address` \n'
            'set n.stationName = record.`name` \n'
            'return count(*) as total',
 'affects': 'unwind $records AS record \n'
       

In [17]:

for q in cypher_import.keys():
    print("about to import " + q + " from file " + railMappings[q]["@fileName"])
    df = pd.read_csv(railMappings[q]["@fileName"])
    result = insert_data(session, cypher_import[q], df, batch_size = 300) 
    print(result)

for md in mapping_defs:
    session.run(md)

about to import Event from file https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv
QUERY:
 unwind $records AS record 
merge (n:Event { `eventId`: record.`id`} ) 
set n.eventDescription = record.`desc` 
set n.eventType = record.`type` 
return count(*) as total
---------------------------


C:\Users\Portfolio-Resolution\AppData\Local\Temp\ipykernel_11812\2121215017.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,


{'total': 143, 'batches': 1, 'time': 4.165135622024536}
{'total': 143, 'batches': 1, 'time': 4.165135622024536}
about to import Station from file https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-stations-all.csv
QUERY:
 unwind $records AS record 
merge (n:Station { `stationCode`: record.`crs`} ) 
set n.lat = record.`lat` 
set n.long = record.`long` 
set n.stationAddress = record.`address` 
set n.stationName = record.`name` 
return count(*) as total
---------------------------
{'total': 300, 'batches': 1, 'time': 0.1125643253326416}


C:\Users\Portfolio-Resolution\AppData\Local\Temp\ipykernel_11812\2121215017.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,
C:\Users\Portfolio-Resolution\AppData\Local\Temp\ipykernel_11812\2121215017.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,


{'total': 600, 'batches': 2, 'time': 0.3295478820800781}
{'total': 900, 'batches': 3, 'time': 0.5256023406982422}


...

{'total': 5381, 'batches': 18, 'time': 18.809684991836548}


C:\Users\Portfolio-Resolution\AppData\Local\Temp\ipykernel_11812\2121215017.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,


{'total': 5681, 'batches': 19, 'time': 19.772650957107544}


C:\Users\Portfolio-Resolution\AppData\Local\Temp\ipykernel_11812\2121215017.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  res = session.write_transaction( lambda tx: tx.run(query,


{'total': 5782, 'batches': 20, 'time': 20.115660667419434}
{'total': 5782, 'batches': 20, 'time': 20.115660667419434}
